In [2]:

!apt-get update
!apt-get install -y nmap
!pip install python-nmap

import nmap
import socket
import sys
import csv
from datetime import datetime

# Function to get IP address from hostname
def get_ip_address(host):
    try:
        ip = socket.gethostbyname(host)
        print(f"Resolved {host} to IP: {ip}")
        return ip
    except socket.gaierror:
        print(f"Error: Could not resolve hostname {host}")
        sys.exit(1)

# Function to validate port range
def validate_port_range(port_range):
    try:
        if '-' in port_range:
            start, end = map(int, port_range.split('-'))
            if 1 <= start <= end <= 65535:
                return port_range
            else:
                raise ValueError
        else:
            port = int(port_range)
            if 1 <= port <= 65535:
                return f"{port}-{port}"
            else:
                raise ValueError
    except ValueError:
        print("Invalid port range. Using default range 1-1000.")
        return "1-1000"

# Function to perform network scan
def network_scan(target_ip, port_range="1-1000", scan_type="-sS"):
    try:
        # Initialize Nmap PortScanner
        nm = nmap.PortScanner()

        # Start scan
        print(f"\nStarting scan on {target_ip} at {datetime.now()}")
        nm.scan(hosts=target_ip, ports=port_range, arguments=scan_type + " -sV")

        # Process scan results
        open_ports = []
        for host in nm.all_hosts():
            print(f"\nHost: {host} ({nm[host].hostname()})")
            print(f"State: {nm[host].state()}")

            for proto in nm[host].all_protocols():
                print(f"\nProtocol: {proto}")
                ports = nm[host][proto].keys()

                for port in sorted(ports):
                    state = nm[host][proto][port]['state']
                    if state == 'open':
                        service = nm[host][proto][port]['name']
                        version = nm[host][proto][port].get('version', 'unknown')
                        open_ports.append(port)
                        print(f"Port: {port}\tState: {state}\tService: {service}\tVersion: {version}")

                        # Basic vulnerability check
                        known_vulnerabilities = {
                            'http': 'Possible CVE-2021-41773 (Apache < 2.4.51)',
                            'ftp': 'Possible CVE-2018-1270 (Anonymous FTP access)',
                            'telnet': 'Possible CVE-2017-0144 (Telnet vulnerabilities)',
                            'smb': 'Possible CVE-2017-0143 (SMBv1 exploits, e.g., EternalBlue)'
                        }
                        if service in known_vulnerabilities:
                            print(f"⚠️ {known_vulnerabilities[service]} on port {port}")
                        elif 'anonymous' in version.lower():
                            print(f"⚠️ Potential Vulnerability: Anonymous access detected on {service} at port {port}")

        # Summary
        print(f"\nScan Summary for {target_ip}:")
        print(f"Open Ports Found: {len(open_ports)}")
        print(f"Ports: {open_ports}")
        print(f"Scan completed at {datetime.now()}")

        # Save to CSV
        if nm.all_hosts():
            with open('scan_results.csv', 'w', newline='') as csvfile:
                writer = csv.writer(csvfile)
                writer.writerow(['Host', 'Port', 'State', 'Service', 'Version'])
                for host in nm.all_hosts():
                    for proto in nm[host].all_protocols():
                        for port in nm[host][proto].keys():
                            writer.writerow([
                                host,
                                port,
                                nm[host][proto][port]['state'],
                                nm[host][proto][port]['name'],
                                nm[host][proto][port].get('version', 'unknown')
                            ])
            print("Results saved to 'scan_results.csv'")

        return open_ports, nm

    except Exception as e:
        print(f"Error during scan: {str(e)}")
        return [], None

# Main function to run the scanner
def main():
    # Define target
    target = input("Enter target hostname or IP (e.g., scanme.nmap.org): ").strip()

    # Resolve hostname to IP
    target_ip = get_ip_address(target)

    # Define port range
    port_range = input("Enter port range (e.g., 1-1000) or press Enter for default (1-1000): ").strip()
    if not port_range:
        port_range = "1-1000"
    else:
        port_range = validate_port_range(port_range)

    # Perform scan
    open_ports, nm = network_scan(target_ip, port_range)

    # Save results to text file
    with open('scan_results.txt', 'w') as f:
        f.write(f"Scan Results for {target_ip} at {datetime.now()}\n")
        f.write(f"Open Ports: {open_ports}\n")
        if nm:
            f.write(f"Full Nmap Output:\n{nm.command_line()}\n")
    print("\nResults saved to 'scan_results.txt'")

if __name__ == "__main__":
    main()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
Reading package lists... Done
Building dependency tree... Done
Reading